# User Based Walk Through - Random User - MAE

In [5]:
#Read the data

import numpy as np
import pandas as pd
from sklearn import *
import final_functions as our_funcs
import warnings 
warnings.filterwarnings('ignore')


# !! Just read in data_df as array or dataframe
data_df = pd.read_csv("Clothing_Shoes_Jewelry_5.csv")
data_df = data_df.drop('Unnamed: 0', 1)

#Getting unique items
unique_products = data_df.asin.unique()

#Get metadata
metadata = pd.read_csv("clothing_shoes_jewelry_metadata.csv")
metadata = metadata.drop('Unnamed: 0', 1)
#data_df_small = data_df.ix[0:4000,:]

# !! Just read in nonan_test as a numpy array (possible?)
#If not, here's the work:
data_df_slim = data_df[['asin', 'overall', 'reviewerID']]
user_item_mat = data_df_slim.pivot(index='reviewerID', columns = 'asin', values='overall')
test = np.array(user_item_mat)
nonan_test = np.nan_to_num(test)

In [6]:
#Creating user similarity matrix
user_sim_mat = metrics.pairwise.cosine_similarity(nonan_test)

In [7]:
#Creating item similarity matrix
nonan_test_t = nonan_test.T
itm_sim_mat = metrics.pairwise.cosine_similarity(nonan_test_t)

In [9]:
print("Welcome to the Amazon clothing, shoes & jewelry recommender")
print('\n')
random_or_input = input("Would you like to review some products and get recommendations, or see recommendations for a random user? \n (type 'User input' or 'Random user'): ")
print('\n')
print("You selected: ", random_or_input)

if random_or_input == 'User input':
    user_rev = [0]*np.shape(nonan_test)[1]
    for i in range(5):
        i += 1
        random_prod = np.random.randint(0, np.shape(nonan_test)[1])
        asin_pd = unique_products[random_prod]
        product = metadata[metadata['asin']==asin_pd].title.item()# 'categories']]
        product = str(product)
        user_review = float(input("Please rate " + product + ": "))
        user_rev[random_prod] = user_review
    
    np.insert(nonan_test, len(nonan_test), user_rev, 0)
    #serve up random rows and get 5 reviews for 5 prods
    #result: a list of same length as array's width: w/ reviews at the correct spot based on column index
    u1 = np.shape(nonan_test)[0]-1
    nonan_test = np.append(nonan_test, np.array([user_rev]), axis = 0)               
                    

else:
    random_row = np.random.randint(0, np.shape(nonan_test)[0])
    u1 = random_row

print(u1)
unrated, rated = our_funcs.getUnrated(nonan_test,u1)
#print info about the user: "product name: review"
for item in rated:
    asin_pd = unique_products[item]
    product = metadata[metadata['asin']==asin_pd].title.item()# 'categories']]
    product = str(product)
    print("User Rated:" + product)

Welcome to the Amazon clothing, shoes & jewelry recommender


Would you like to review some products and get recommendations, or see recommendations for a random user? 
 (type 'User input' or 'Random user'): Random user


You selected:  Random user
21696
User Rated:KEEN Women's Newport H2 Sandal
User Rated:Sloggers Women's  Premium Garden Clog with Premium Insole Insole,  Geisha Blue  - Wo's size 9 - Style 260GB09
User Rated:Sloggers  Women's Rain and Garden Boot with &quot;All-Day-Comfort&quot; Insole, Midsummer Blue Print - Wo's size 10 - Style 5002BL10
User Rated:The &quot;Summit Collection&quot; Pacific Poncho from Charles River Apparel
User Rated:Sloggers  Women's Rain and Garden Shoe with &quot;All-Day-Comfort&quot; Insole, Midsummer Black Print - Wo's size 6 - Style 5102BK06
User Rated:WonderWink Women's Scrubs Romeo 6 Pocket Flare Leg Pant
User Rated:Roamans Women's Plus Size Kate Bigshirt
User Rated:Salomon Women's XR Mission Running Shoe
User Rated:Women's Plus Size Pants, cr

In [13]:
#Let responder give rec method
rec_method = input("Select recommendation method \n(type 'User-based', 'Item-based', 'Hybrid'): ")
num_recs = int(input("Number of products to recommend? (Input an integer:\n )"))
print('\n')
#Three branches, based on input
#User-based branch
if rec_method == 'User-based':
    n = int(input("How many similar users to base rec's off of? (Input an integer:\n )"))
    print('\n')
    simUsers = our_funcs.getUserPC(user_sim_mat,u1,n)
    #Serve up the rec's

    our_funcs.userBasedRecs(nonan_test, simUsers, unrated, num_recs, 
                            rated, unique_products, metadata)

#Item-based branch
elif rec_method == 'Item-based':
    
    our_funcs.itemSim(nonan_test, u1, num_recs, unique_products, metadata, itm_sim_mat, unrated, rated)
    
#Hybrid branch
elif rec_method == 'Hybrid':
    n = int(input("How many similar users to base rec's off of? (Input an integer:\n )"))
    simUsers = our_funcs.getUserPC(user_sim_mat,u1,n)
    
    our_funcs.hybridRec(nonan_test, simUsers, unrated, num_recs, 
              rated, unique_products, metadata)


Select recommendation method 
(type 'User-based', 'Item-based', 'Hybrid'): User-based
Number of products to recommend? (Input an integer:
 )3


How many similar users to base rec's off of? (Input an integer:
 )3


User Rated: KEEN Women's Newport H2 Sandal
User Rated: Sloggers Women's  Premium Garden Clog with Premium Insole Insole,  Geisha Blue  - Wo's size 9 - Style 260GB09
User Rated: Sloggers  Women's Rain and Garden Boot with &quot;All-Day-Comfort&quot; Insole, Midsummer Blue Print - Wo's size 10 - Style 5002BL10
User Rated: The &quot;Summit Collection&quot; Pacific Poncho from Charles River Apparel
User Rated: Sloggers  Women's Rain and Garden Shoe with &quot;All-Day-Comfort&quot; Insole, Midsummer Black Print - Wo's size 6 - Style 5102BK06
User Rated: WonderWink Women's Scrubs Romeo 6 Pocket Flare Leg Pant
User Rated: Roamans Women's Plus Size Kate Bigshirt
User Rated: Salomon Women's XR Mission Running Shoe
User Rated: Women's Plus Size Pants, cropped length in cool linen blend

In [15]:
# Validation of results

print("Now that you've seen some examples, let's talk about performance \n")
print("-- Recommender evaluator -- \n\n")

rec_method = input("Which recommendation method do you want to evaluate? \n(type 'User-based', 'Item-based', 'Hybrid'): ")

if rec_method == 'User-based' or rec_method == 'Hybrid':
    n = int(input("How many similar users to base rec's off of? (type an integer)\n"))
    eval_method = input("How do you want to evaluate the recommender system? \n(type 'MAE' or 'Recall')")
    num_recs = int(input("How many recommendations to give?(type an integer)\n"))
    print('\n')
    #User branch
    if rec_method == "User-based":
        if eval_method == 'MAE':
            our_funcs.calc_mae_user(nonan_test, 0.01, n, user_sim_mat)
        elif eval_method == 'Recall':
            # !! needs work 
            our_funcs.user_based_recall(nonan_test, 0.001, n, num_recs, unique_products, metadata, user_sim_mat)
    
    #Hybrid branch
    elif rec_method == 'Hybrid':
        if eval_method == 'MAE':
            our_funcs.calc_mae_hyb(nonan_test, 0.01, n, unique_products, metadata, user_sim_mat)
        elif eval_method == 'Recall':
            # !! needs work 
            our_funcs.hybrid_recall(nonan_test, 0.01, n, num_recs, unique_products, metadata, user_sim_mat)
            
elif rec_method == 'Item-based':
    eval_method = input("How do you want to evaluate the recommender system? \n(type 'MAE' or 'Recall')")
    num_recs = int(input("How many recommendations to give? \n(type an integer): "))
    
    if eval_method == 'MAE':
        our_funcs.calc_mae_itm(nonan_test, 0.01, n, itm_sim_mat)
        
    elif eval_method == 'Recall':
        our_funcs.item_based_recall(nonan_test, u1, 0.01, n, num_recs, unique_products, metadata, itm_sim_mat)
        

Now that you've seen some examples, let's talk about performance 

-- Recommender evaluator -- 


Which recommendation method do you want to evaluate? 
(type 'User-based', 'Item-based', 'Hybrid'): User-based
How many similar users to base rec's off of? (type an integer)
3
How do you want to evaluate the recommender system? 
(type 'MAE' or 'Recall')MAE
How many recommendations to give?(type an integer)
3


MEAN ABSOLUTE ERROR:  0.717557251908
